In [1]:
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup

In [20]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=1900-01-01&endtime=2020-10-22&minlatitude=34.9884&maxlatitude=36.6871&minlongitude=-90.3131&maxlongitude=-81.6518'

In [21]:
r = requests.get(url)

In [22]:
print(r.text[:1000])

time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
2020-10-21T21:49:45.720Z,36.0705,-89.8023333,6.84,2.14,md,46,40,0.0643,0.1,nm,nm60312762,2020-10-22T12:49:53.160Z,"2 km NNE of Cooter, Missouri",earthquake,0.17,0.49,0.092,31,reviewed,nm,nm
2020-10-21T20:28:42.930Z,36.525,-89.5691667,6.85,1.69,md,19,50,0.02596,0.03,nm,nm60312752,2020-10-22T14:45:56.660Z,"3 km E of Marston, Missouri",earthquake,0.34,0.63,0.167,12,reviewed,nm,nm
2020-10-21T04:55:51.350Z,36.5456667,-89.649,8.01,1.61,md,24,44,0.02245,0.04,nm,nm60312697,2020-10-21T13:04:57.100Z,"4 km NW of Marston, Missouri",earthquake,0.44,0.4,0.147,21,reviewed,nm,nm
2020-10-21T03:00:08.800Z,36.3136667,-89.5361667,9.5,1.23,md,17,65,0.02693,0.06,nm,nm60312692,2020-10-21T13:18:24.430Z,"7 km NW of Ridgely, Tennessee",earthquake,0.68,0.51,0.032,10,reviewed,nm,nm
2020-10-20T18:50:21.800Z,36.2675,-89.5233333,8.84,2.33,md,46,33,0.0152,0.11

In [23]:
type(r)

requests.models.Response

In [24]:
r.status_code

200

In [25]:
#did not work
usgs = pd.read_csv(StringIO(r.text))

In [26]:
#usgs = pd.read_csv(r.url, header=None, sep='\n')
#usgs = usgs[0].str.split(',', expand=True)
usgs = pd.read_csv(r.url)
usgs.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-10-21T21:49:45.720Z,36.070500,-89.802333,6.84,2.14,md,46.0,40.0,0.06430,0.10,...,2020-10-22T12:49:53.160Z,"2 km NNE of Cooter, Missouri",earthquake,0.17,0.49,0.092,31.0,reviewed,nm,nm
1,2020-10-21T20:28:42.930Z,36.525000,-89.569167,6.85,1.69,md,19.0,50.0,0.02596,0.03,...,2020-10-22T14:45:56.660Z,"3 km E of Marston, Missouri",earthquake,0.34,0.63,0.167,12.0,reviewed,nm,nm
2,2020-10-21T04:55:51.350Z,36.545667,-89.649000,8.01,1.61,md,24.0,44.0,0.02245,0.04,...,2020-10-21T13:04:57.100Z,"4 km NW of Marston, Missouri",earthquake,0.44,0.40,0.147,21.0,reviewed,nm,nm
3,2020-10-21T03:00:08.800Z,36.313667,-89.536167,9.50,1.23,md,17.0,65.0,0.02693,0.06,...,2020-10-21T13:18:24.430Z,"7 km NW of Ridgely, Tennessee",earthquake,0.68,0.51,0.032,10.0,reviewed,nm,nm
4,2020-10-20T18:50:21.800Z,36.267500,-89.523333,8.84,2.33,md,46.0,33.0,0.01520,0.11,...,2020-10-21T12:29:35.220Z,"3 km W of Ridgely, Tennessee",earthquake,0.15,0.30,0.142,34.0,reviewed,nm,nm


In [27]:
usgs.tail()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
9158,1916-02-21T23:39:00.000Z,35.5,-82.5,NaN,5.2,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,North Carolina,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
9159,1915-12-07T18:40:00.000Z,36.0,-90.0,NaN,4.5,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Arkansas,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,bar
9160,1913-03-28T21:50:00.000Z,36.2,-83.7,NaN,4.1,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,eastern Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
9161,1903-11-04T19:14:00.000Z,36.5,-89.8,NaN,5.1,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Missouri,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
9162,1903-11-04T18:18:00.000Z,36.5,-89.5,NaN,4.6,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc


In [28]:
usgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9163 entries, 0 to 9162
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             9163 non-null   object 
 1   latitude         9163 non-null   float64
 2   longitude        9163 non-null   float64
 3   depth            9150 non-null   float64
 4   mag              9160 non-null   float64
 5   magType          9152 non-null   object 
 6   nst              6155 non-null   float64
 7   gap              6153 non-null   float64
 8   dmin             5979 non-null   float64
 9   rms              6164 non-null   float64
 10  net              9163 non-null   object 
 11  id               9163 non-null   object 
 12  updated          9163 non-null   object 
 13  place            9163 non-null   object 
 14  type             9163 non-null   object 
 15  horizontalError  6214 non-null   float64
 16  depthError       6213 non-null   float64
 17  magError      

In [30]:
#usgs[usgs['place'].isin(['Tennessee'])]

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
67,2020-09-05T03:41:09.170Z,35.700167,-84.348167,14.83,1.98,md,16.0,81.0,0.1258,0.11,...,2020-10-14T23:10:43.040Z,Tennessee,earthquake,0.27,0.58,0.195,13.0,reviewed,se,se
5552,2001-07-08T03:05:11.100Z,35.880000,-86.800000,0.00,2.60,md,10.0,NaN,NaN,NaN,...,2014-11-07T01:13:15.794Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,tva,tva
6013,1998-03-18T20:22:08.200Z,35.200000,-84.200000,5.00,2.50,mblg,NaN,NaN,NaN,NaN,...,2014-11-07T01:04:46.683Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,macr,us
6235,1995-06-06T21:27:11.800Z,36.220000,-89.470000,5.00,3.10,md,NaN,NaN,NaN,NaN,...,2014-11-07T00:58:06.696Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,slm,slm
6236,1995-05-27T19:51:10.400Z,36.170000,-89.430000,6.40,3.00,md,NaN,NaN,NaN,NaN,...,2014-11-07T00:58:00.398Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,slm,slm
7804,1982-01-02T02:00:25.760Z,35.190000,-86.440000,7.00,2.90,mblg,NaN,NaN,NaN,NaN,...,2014-11-07T00:30:36.803Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,tei,us
8971,1975-05-14T23:03:05.900Z,35.947000,-85.249000,5.00,2.70,lg,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:30.327Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,us,slm
9131,1974-03-12T12:30:28.600Z,35.660000,-89.790000,5.00,3.20,lg,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:23.314Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,j,slm
9132,1974-03-10T04:34:19.700Z,36.210000,-89.530000,5.00,2.50,lg,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:23.286Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,j,slm
9133,1974-01-08T01:12:37.400Z,36.200000,-89.390000,1.00,4.10,mb,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:22.320Z,Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,j,us


In [ ]:
#usgs = pd.read_csv(r.url, header=None, sep='\n')
#usgs = usgs[0].str.split(',', expand=True)
#usgs.head()


In [ ]:
#html_doc = r.text